In [30]:
import pandas as pd
import numpy as np
import datetime
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cases= r'I:\Market Planning\Revenue and Sales Analysis\Individual Folders\Janine\resources\Python\Covid\RAW_global_confirmed_cases.csv'
deaths= r'I:\Market Planning\Revenue and Sales Analysis\Individual Folders\Janine\resources\Python\Covid\RAW_global_deaths.csv'

In [3]:
df_cases = pd.read_csv(cases, usecols=['Country/Region', 'Lat', 'Long', '11/13/20'], index_col=0, converters={'11/13/20':float})
pd.options.display.float_format = '{:,.2f}'.format
df_cases = df_cases.rename(columns={'11/13/20':'Cases'})
df_cases = df_cases.groupby(by=['Country/Region']).sum()
df_cases

,Lat,Long,Cases
Country/Region,,,
Afghanistan,33.94,67.71,"42,969.00"
Albania,41.15,20.17,"26,701.00"
Algeria,28.03,1.66,"65,975.00"
Andorra,42.51,1.52,"5,725.00"
Angola,-11.20,17.87,"13,228.00"
...,...,...,...
West Bank and Gaza,31.95,35.23,"61,514.00"
Western Sahara,24.22,-12.89,10.00
Yemen,15.55,48.52,"2,072.00"


In [4]:
df_deaths = pd.read_csv(deaths, usecols=['Country/Region', 'Lat', 'Long', '11/13/20'], index_col=0, converters={'11/13/20':float})
pd.options.display.float_format = '{:,.2f}'.format
df_deaths = df_deaths.rename(columns={'11/13/20':'Deaths'})
df_deaths = df_deaths.groupby(by=['Country/Region']).sum()
df_deaths

,Lat,Long,Deaths
Country/Region,,,
Afghanistan,33.94,67.71,"1,595.00"
Albania,41.15,20.17,605.00
Algeria,28.03,1.66,"2,124.00"
Andorra,42.51,1.52,75.00
Angola,-11.20,17.87,317.00
...,...,...,...
West Bank and Gaza,31.95,35.23,548.00
Western Sahara,24.22,-12.89,1.00
Yemen,15.55,48.52,605.00


In [5]:
df_sum = df_cases.merge(df_deaths['Deaths'], on='Country/Region', how='left').reset_index()
df_sum['Mortality'] = round((df_sum['Deaths']/df_sum['Cases'])*100,2)
df_sum

,Country/Region,Lat,Long,Cases,Deaths,Mortality
0,Afghanistan,33.94,67.71,"42,969.00","1,595.00",3.71
1,Albania,41.15,20.17,"26,701.00",605.00,2.27
2,Algeria,28.03,1.66,"65,975.00","2,124.00",3.22
3,Andorra,42.51,1.52,"5,725.00",75.00,1.31
4,Angola,-11.20,17.87,"13,228.00",317.00,2.40
...,...,...,...,...,...,...
186,West Bank and Gaza,31.95,35.23,"61,514.00",548.00,0.89
187,Western Sahara,24.22,-12.89,10.00,1.00,10.00
188,Yemen,15.55,48.52,"2,072.00",605.00,29.20
189,Zambia,-13.13,27.85,"17,093.00",350.00,2.05


In [6]:
print(df_sum.columns)
print(type(df_sum))

Index(['Country/Region', 'Lat', 'Long', 'Cases', 'Deaths', 'Mortality'], dtype='object')
<class 'pandas.core.frame.DataFrame'>


In [7]:
alt.Chart(df_sum).mark_circle(size=60).encode(
    x='Deaths',
    y='Cases',
    color='Deaths',
    size='Deaths',
    tooltip=['Country/Region', 'Cases', 'Deaths', 'Mortality']
).interactive()

alt.Chart(...)

In [8]:
# Top 20 countries with highest number of deaths

alt.Chart(
    df_sum,
).mark_bar().encode(
    x=alt.X('Country/Region:N', sort='-y'),
    y=alt.Y('Deaths'),
    color=alt.Color('Deaths'),
    tooltip=['Deaths','Cases','Mortality'],

).transform_window(
    rank='rank(Deaths)',
    sort=[alt.SortField('Deaths', order='descending')]
).transform_filter(
    (alt.datum.rank < 21)
)

alt.Chart(...)

In [9]:
# Top 20 countries with highest mortality

alt.Chart(
    df_sum,
).mark_bar().encode(
    x=alt.X('Country/Region:N', sort='-y'),
    y=alt.Y('Mortality'),
    color=alt.Color('Mortality'),
    tooltip=['Deaths','Cases','Mortality'],

).transform_window(
    rank='rank(Mortality)',
    sort=[alt.SortField('Mortality', order='descending')]
).transform_filter(
    (alt.datum.rank < 21)
)

alt.Chart(...)

In [10]:
# sort tables by descending number of deaths or cases
df_top_deaths = df_sum.sort_values(by='Deaths', ascending=False)
df_top_deaths.head(10)

,Country/Region,Lat,Long,Cases,Deaths,Mortality
176,US,40.00,-100.00,"10,737,335.00","244,332.00",2.28
23,Brazil,-14.23,-51.93,"5,810,652.00","164,737.00",2.84
79,India,20.59,78.96,"8,773,479.00","129,188.00",1.47
114,Mexico,23.63,-102.55,"997,393.00","97,624.00",9.79
180,United Kingdom,270.03,-482.92,"1,321,031.00","51,396.00",3.89
85,Italy,41.87,12.57,"1,107,303.00","44,139.00",3.99
62,France,91.40,60.04,"1,915,677.00","42,600.00",2.22
160,Spain,40.46,-3.75,"1,458,591.00","40,769.00",2.80
81,Iran,32.43,53.69,"738,322.00","40,582.00",5.50
135,Peru,-9.19,-75.02,"930,237.00","35,067.00",3.77


In [124]:
df_daily_cases = pd.read_csv(cases, index_col=0)
df_daily_cases = df_daily_cases.drop(columns=['Province/State','Lat','Long'])
df_daily_cases = df_daily_cases.groupby(by='Country/Region').sum()
df_daily_cases

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,41935,41975,42033,42092,42297,42463,42609,42795,42969,43035
Albania,0,0,0,0,0,0,0,0,0,0,...,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233
Algeria,0,0,0,0,0,0,0,0,0,0,...,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819
Andorra,0,0,0,0,0,0,0,0,0,0,...,5135,5135,5319,5383,5437,5477,5567,5616,5725,5725
Angola,0,0,0,0,0,0,0,0,0,0,...,12102,12223,12335,12433,12680,12816,12953,13053,13228,13374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,56672,57226,57657,58158,58838,59422,60065,60784,61514,62167
Western Sahara,0,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
Yemen,0,0,0,0,0,0,0,0,0,0,...,2063,2067,2070,2070,2071,2071,2071,2071,2072,2072


In [103]:
print(type(df_daily_cases))

<class 'pandas.core.frame.DataFrame'>


In [112]:
df_transposed = df_daily_cases.transpose().reset_index()
df_transposed

Country/Region,index,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
0,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
3,1/25/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,1/26/20,0,0,0,0,0,0,0,0,4,...,0,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,11/10/20,42463,25294,63446,5477,12816,131,1262476,108687,27671,...,3620,69027,1,95445,1226,59422,10,2071,16997,8610
294,11/11/20,42609,25801,64257,5567,12953,131,1273356,110548,27675,...,3700,69397,1,95750,1252,60065,10,2071,17036,8667
295,11/12/20,42795,26211,65108,5616,13053,131,1284519,112680,27676,...,3795,69560,1,96140,1253,60784,10,2071,17056,8696
296,11/13/20,42969,26701,65975,5725,13228,133,1296378,114383,27682,...,3883,69754,1,96441,1256,61514,10,2072,17093,8765


In [12]:
countries = df_sum['Country/Region'].to_list
countries

<bound method IndexOpsMixin.tolist of 0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
186    West Bank and Gaza
187        Western Sahara
188                 Yemen
189                Zambia
190              Zimbabwe
Name: Country/Region, Length: 191, dtype: object>

In [154]:
df_final=pd.melt(df_transposed, id_vars='index')
df_final = df_final.rename(columns={'index':'Date','value':'Cases'})
df_final['Date']= pd.to_datetime(df_final['Date'])
df_final

,Date,Country/Region,Cases
0,2020-01-22,Afghanistan,0
1,2020-01-23,Afghanistan,0
2,2020-01-24,Afghanistan,0
3,2020-01-25,Afghanistan,0
4,2020-01-26,Afghanistan,0
...,...,...,...
56913,2020-11-10,Zimbabwe,8610
56914,2020-11-11,Zimbabwe,8667
56915,2020-11-12,Zimbabwe,8696
56916,2020-11-13,Zimbabwe,8765


In [158]:
df_chart=df_final.loc[(df_final['Country/Region']=="India")|
                      (df_final['Country/Region']=="US")|
                      (df_final['Country/Region']=="Brazil")|
                      (df_final['Country/Region']=="Mexico")|
                      (df_final['Country/Region']=="China")|
                      (df_final['Country/Region']=="Italy")|
                      (df_final['Country/Region']=="France")|
                      (df_final['Country/Region']=="Germany")
                     ]
df_chart = alt.Chart(df_chart).mark_line().encode(
    x='Date',
    y='Cases',
    color='Country/Region',
    tooltip=['Country/Region','Cases']
).interactive()
df_chart

alt.Chart(...)

In [159]:
df_chart.save('df_chart.html')